In [ ]:
%%capture
!apt-get install poppler-utils tesseract-ocr tesseract-ocr-por
!pip install PyMuPDF pytesseract pdf2image opencv-python

In [ ]:
import fitz  # PyMuPDF
import pytesseract
import cv2
import os
import numpy as np
from pdf2image import convert_from_path
from textblob import TextBlob

## Captura dos Arquivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
legislacao_path = '/content/drive/MyDrive/_Graduacao/periodo7/NLP/pln_trabFinal_agataBrazao_ricardoMiranda/legislacao/'
legislacao_pdfs = os.listdir(legislacao_path)
print(legislacao_pdfs)

['res_18_2007_ativ_complemtares.pdf', 'port_05_2020_aproveit_disci_do_pibid.pdf', 'resolução_021_2007_consepe_aproveit_estudos.pdf', 'res_23_2017_regime_didático.pdf', 'res_070_2011_afinidade_cursos.pdf', 'res_38_2015_aceleração_de_estudos.pdf', 'res_10_2016_disciplinas_presenciais.pdf', 'res_18_93_créd_optativos.pdf', 'res_2_2015_cne_diretcurric_nacionais.pdf', 'res_32_2016_boletim_diário_classe.pdf', 'port_2304_95_nada_consta.pdf', 'res_69_2010_curso_de_férias.pdf', 'res_20_2019_criação_ext_modificação_curricular.pdf', 'lei_carteira_estudantil.pdf', 'port_048_2019_heteroidentificação.pdf', 'estatuto_da_ufam.pdf', 'estágio_de_graduação.pdf', 'res_31_2015_entrega_revisão_exerc_e_provas.pdf', 'portaria_no_1.095_de_25_de_outubro_de_2018_imprensa_nacional.pdf', 'resolução_027_2010_enade.pdf', 'resolução_cne_diretcurricnacionais_out2019.pdf', 'resolução_081_2007_consuni.pdf', 'resolução_010_2007_consad.pdf', 'resolução_010_2004_consuni.pdf', 'portaria_301_1998.p

## Funções

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
# tesseract para português
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
tessdata_dir_config = r'--tessdata-dir "/usr/share/tesseract-ocr/4.00/tessdata/" -l por'

def preprocess_image(image):
    # escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # desfocagem
    blurred = cv2.medianBlur(gray, 3)
    # binarização
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return binary


def extract_text_from_scanned_pdf(pdf_path):
    # Converter PDF para imagens
    images = convert_from_path(pdf_path)
    text = ""
    for image in images:
        # Converter a imagem PIL para um array numpy
        open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        # Pré-processar a imagem
        preprocessed_image = preprocess_image(open_cv_image)
        # Extrair texto usando Tesseract
        extracted_text = pytesseract.image_to_string(preprocessed_image, config=tessdata_dir_config)
        # Adicionar o texto extraído ao resultado final
        text += extracted_text + "\n"

    return text


In [ ]:
def is_scanned_pdf(file_path):
    doc = fitz.open(file_path)
    has_text = False
    has_images = False

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        if text.strip():
            has_text = True
        images = page.get_images(full=True)
        if images:
            has_images = True

    return not has_text and has_images

def extract_text(pdf_path):
  print("Iniciando verificação...")
  if is_scanned_pdf(pdf_path):
    print("Iniciando extração de PDF Scanneado...")
    return extract_text_from_scanned_pdf(pdf_path)
    print("PDF Scanneado Extraído!")
  else:
    print("Iniciando extração de PDF Não Scanneado...")
    return extract_text_from_pdf(pdf_path)
    print("PDF Não Scanneado Extraído!")


>> .

In [ ]:
text = extract_text_from_scanned_pdf("/content/res_32_2016_boletim_diário_classe.pdf")
with open("/content/res_32_2016_boletim_diário_classe.txt", 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [ ]:
pdf_path = legislacao_path+legislacao_pdfs[41]
text = extract_text(pdf_path)

Iniciando verificação...
Iniciando extração de PDF Scanneado...


In [ ]:
def linha(line):
  new_line = ''

  cond = False
  for letter in line:
    if letter == "º":
      cond = True
    elif cond:
      new_line += letter

  return new_line

In [ ]:
def clean_text(text):
  text = text.split('\n')

  list_text = []

  for line in text:
    if line == '' or line =='\x0c' or line== ' ':
      continue
    list_text.append(line)

  text = '\n'.join(list_text)

  return text

In [ ]:
a = "a aventura\n vai começar\n\ntodos juntos vamos visitar\n\n \n \n aaaaaaaaaaaaaaaa"

print(clean_text(a))

a aventura
 vai começar
todos juntos vamos visitar
 aaaaaaaaaaaaaaaa


In [ ]:
count = 1
for pdf in legislacao_pdfs:
    print(f"{count}/60")
    pdf_path = legislacao_path + pdf
    text = extract_text(pdf_path)
    # lógica para adicionar o texto em um arquivo .txt
    txt_filename = f"/content/textos/{pdf[:-4]}.txt"
    txt_path = os.path.join(legislacao_path, txt_filename)
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

    count += 1

1/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
2/60
Iniciando verificação...
Iniciando extração de PDF Não Scanneado...
3/60
Iniciando verificação...
Iniciando extração de PDF Não Scanneado...
4/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
5/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
6/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
7/60
Iniciando verificação...
Iniciando extração de PDF Não Scanneado...
8/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
9/60
Iniciando verificação...
Iniciando extração de PDF Não Scanneado...
10/60
Iniciando verificação...
Iniciando extração de PDF Não Scanneado...
11/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
12/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
13/60
Iniciando verificação...
Iniciando extração de PDF Scanneado...
14/60
Iniciando verificação...
Iniciando extração de PDF Não Scanneado...
15/60